# Import

In [30]:
import pandas as pd

In [31]:
original_csv = pd.read_csv('Sample-Superstore.csv', encoding='windows-1252')

In [32]:
original_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [33]:
original_csv.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Cleaning

### Columns: lowercase, no space + new column with "Supplier Price"

In [34]:
updated_csv = original_csv
# updated_csv = updated_csv.rename(str.l, axis=1)
updated_csv.columns
updated_csv.columns = [x.replace(' ', '') for x in updated_csv.columns]
updated_csv.columns = [x.replace('-', '') for x in updated_csv.columns]
updated_csv.columns
# updated_csv['supplier_price'] = updated_csv['sales'] / updated_csv['quantity'] - updated_csv['profit'] / updated_csv['quantity'] - updated_csv['sales'] * updated_csv['discount']

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [35]:
updated_csv["SupplierPrice"] = (
    (updated_csv["Sales"] * (1 - updated_csv["Discount"]) * updated_csv["Quantity"] - updated_csv["Profit"])
    / updated_csv["Quantity"]
)
updated_csv.rename(columns={'Sales':'SalePrice'}, inplace=True)
updated_csv.head(20)


,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,Region,ProductID,Category,SubCategory,ProductName,SalePrice,Quantity,Discount,Profit,SupplierPrice
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,241.003200
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,658.746000
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,11.184300
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,603.273825
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,16.636200
5,6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,46.835800
6,7,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656,6.788600
7,8,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152,710.602400
8,9,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825,12.875700
9,10,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700,108.006000


In [36]:
updated_csv.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'SalePrice', 'Quantity', 'Discount', 'Profit', 'SupplierPrice'],
      dtype='object')

### Change column order

In [37]:
new_order = [
    'RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode',
    'CustomerID', 'CustomerName', 'Segment', 'Country', 'City', 'State',
    'PostalCode', 'Region', 'ProductID', 'Category', 'SubCategory',
    'ProductName', 'SupplierPrice', 'Quantity', 'SalePrice', 'Discount', 'Profit'
]

updated_csv = updated_csv[new_order]
updated_csv.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'SupplierPrice', 'Quantity', 'SalePrice', 'Discount', 'Profit'],
      dtype='object')

In [38]:
updated_csv.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,Region,ProductID,Category,SubCategory,ProductName,SupplierPrice,Quantity,SalePrice,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,241.003200,2,261.9600,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",658.746000,3,731.9400,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,11.184300,2,14.6200,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,603.273825,5,957.5775,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,16.636200,2,22.3680,0.20,2.5164


# Create Tables

In [104]:
import sqlite3

conn = sqlite3.connect("Ecommerce.db")
cur = conn.cursor()

create_customers_table = """
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID TEXT PRIMARY KEY,
        CustomerName TEXT,
        Segment TEXT
    )
"""

create_orders_table = """
    CREATE TABLE IF NOT EXISTS Orders (
        OrderID TEXT,
        OrderDate TEXT,
        CustomerID TEXT,
        ProductID TEXT,
        PRIMARY KEY (OrderID, ProductID)
    )
"""

create_shipping_table = """
    CREATE TABLE IF NOT EXISTS Shipping (
        OrderID TEXT PRIMARY KEY,
        OrderDate TEXT,
        ShipDate TEXT, 
        ShipMode TEXT
    )
"""

create_address_table = """
    CREATE TABLE IF NOT EXISTS Address (
        CustomerID TEXT PRIMARY KEY,
        Country TEXT,
        City TEXT,
        State TEXT,
        PostalCode INTEGER,
        Region TEXT
    )
"""

create_products_table = """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID TEXT PRIMARY KEY,
        Category TEXT,
        SubCategory TEXT,
        ProductName TEXT
    )
"""

create_sales_table = """
    CREATE TABLE IF NOT EXISTS Sales (
        OrderID TEXT PRIMARY KEY,
        OrderDate TEXT,
        SupplierPrice REAL,
        SalePrice REAL,
        Quantity REAL,
        Discount REAL,
        Profit REAL
    )
"""

cur.execute(create_customers_table)
cur.execute(create_orders_table)
cur.execute(create_shipping_table)
cur.execute(create_address_table)
cur.execute(create_products_table)
cur.execute(create_sales_table)

cur.close()
conn.close()


# CSV's for Tables

### customers.csv

In [ ]:
customers = updated_csv[["CustomerID", "CustomerName", "Segment"]].copy()
customers = customers.drop_duplicates(subset='CustomerID')
customers.info()





<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, 0 to 9558
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    793 non-null    object
 1   CustomerName  793 non-null    object
 2   Segment       793 non-null    object
dtypes: object(3)
memory usage: 24.8+ KB


In [93]:
customers.to_csv("customers.csv", index=False)


### orders.csv

##### Here we didnt get rid of any duplicates. 

In [ ]:
orders = updated_csv[["OrderID", "OrderDate", "CustomerID", "ProductID"]].copy()
# orders = orders.drop_duplicates(subset='OrderID')
orders.info()
orders.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderID     9994 non-null   object
 1   OrderDate   9994 non-null   object
 2   CustomerID  9994 non-null   object
 3   ProductID   9994 non-null   object
dtypes: object(4)
memory usage: 312.4+ KB


,OrderID,OrderDate,CustomerID,ProductID
0,CA-2016-152156,11/8/2016,CG-12520,FUR-BO-10001798
1,CA-2016-152156,11/8/2016,CG-12520,FUR-CH-10000454
2,CA-2016-138688,6/12/2016,DV-13045,OFF-LA-10000240
3,US-2015-108966,10/11/2015,SO-20335,FUR-TA-10000577
4,US-2015-108966,10/11/2015,SO-20335,OFF-ST-10000760


In [70]:
orders.to_csv("orders.csv", index=False)


### shipping.csv

In [75]:
shipping = updated_csv[["OrderID", "OrderDate","ShipDate", "ShipMode"]].copy()
shipping = shipping.drop_duplicates(subset='OrderID')
shipping.info()
shipping.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 5009 entries, 0 to 9993
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   OrderID    5009 non-null   object
 1   OrderDate  5009 non-null   object
 2   ShipDate   5009 non-null   object
 3   ShipMode   5009 non-null   object
dtypes: object(4)
memory usage: 195.7+ KB


,OrderID,OrderDate,ShipDate,ShipMode
0,CA-2016-152156,11/8/2016,11/11/2016,Second Class
2,CA-2016-138688,6/12/2016,6/16/2016,Second Class
3,US-2015-108966,10/11/2015,10/18/2015,Standard Class
5,CA-2014-115812,6/9/2014,6/14/2014,Standard Class
12,CA-2017-114412,4/15/2017,4/20/2017,Standard Class


In [76]:
shipping.to_csv("shipping.csv", index=False)


### address.csv

In [80]:
address = updated_csv[["CustomerID", "Country", "City", "State", "PostalCode", "Region"]].copy()
address = address.drop_duplicates(subset="CustomerID")
address.info()
address.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, 0 to 9558
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  793 non-null    object
 1   Country     793 non-null    object
 2   City        793 non-null    object
 3   State       793 non-null    object
 4   PostalCode  793 non-null    int64 
 5   Region      793 non-null    object
dtypes: int64(1), object(5)
memory usage: 43.4+ KB


,CustomerID,Country,City,State,PostalCode,Region
0,CG-12520,United States,Henderson,Kentucky,42420,South
2,DV-13045,United States,Los Angeles,California,90036,West
3,SO-20335,United States,Fort Lauderdale,Florida,33311,South
5,BH-11710,United States,Los Angeles,California,90032,West
12,AA-10480,United States,Concord,North Carolina,28027,South


In [81]:
address.to_csv("address.csv", index=False)


### products.csv

In [82]:
products = updated_csv[["ProductID", "Category", "SubCategory", "ProductName"]].copy()
products = products.drop_duplicates(subset="ProductID")
products.info()
products.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 1862 entries, 0 to 9935
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductID    1862 non-null   object
 1   Category     1862 non-null   object
 2   SubCategory  1862 non-null   object
 3   ProductName  1862 non-null   object
dtypes: object(4)
memory usage: 72.7+ KB


,ProductID,Category,SubCategory,ProductName
0,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase
1,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...
3,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table
4,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System


In [83]:
products.to_csv("products.csv", index=False)


### sales.csv

In [84]:
sales = updated_csv[["OrderID", "OrderDate", "SupplierPrice", "SalePrice", "Quantity", "Discount", "Profit"]].copy()
# sales = sales.drop_duplicates(subset="OrderID")
sales.info()
sales.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OrderID        9994 non-null   object 
 1   OrderDate      9994 non-null   object 
 2   SupplierPrice  9994 non-null   float64
 3   SalePrice      9994 non-null   float64
 4   Quantity       9994 non-null   int64  
 5   Discount       9994 non-null   float64
 6   Profit         9994 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 546.7+ KB


,OrderID,OrderDate,SupplierPrice,SalePrice,Quantity,Discount,Profit
0,CA-2016-152156,11/8/2016,241.003200,261.9600,2,0.00,41.9136
1,CA-2016-152156,11/8/2016,658.746000,731.9400,3,0.00,219.5820
2,CA-2016-138688,6/12/2016,11.184300,14.6200,2,0.00,6.8714
3,US-2015-108966,10/11/2015,603.273825,957.5775,5,0.45,-383.0310
4,US-2015-108966,10/11/2015,16.636200,22.3680,2,0.20,2.5164


In [85]:
sales.to_csv("sales.csv", index=False)


# Populate Tables

In [106]:
conn = sqlite3.connect("Ecommerce.db")

customers.to_sql(
    "Customers", # the name of the table
    con=conn, # the connection variable
    index=False, # whether to include the index column
    if_exists="append" # define behavior if table already exists
)

conn.close()


In [105]:
conn = sqlite3.connect("Ecommerce.db")

orders.to_sql(
    "Orders", # the name of the table
    con=conn, # the connection variable
    index=False, # whether to include the index column
    if_exists="append" # define behavior if table already exists
)

conn.close()

IntegrityError: UNIQUE constraint failed: Orders.OrderID, Orders.ProductID

In [107]:
## Auto Increment